In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import networkx as nx
np.random.seed(1024)

NUM_STREAM_SPEC = list(range(8, 129, 10))

PERIOD_SPEC = [1, 2, 3, 4, 5, 6]

def period_spec(opt):
    if opt == 1:
        return 2_000_000
    if opt == 2:
        return 400_000
    if opt == 3:
        return int(np.random.choice([500_000, 1_000_000, 2_000_000, 4_000_000]))
    if opt == 4:
        return int(np.random.choice([100_000, 200_000, 400_000, 800_000]))
    if opt == 5:
        return int(np.random.choice([250_000, 500_000, 1_250_000, 2_500_000, 4_000_000]))
    if opt == 6:
        return int(np.random.choice([50_000, 100_000, 250_000, 500_000, 800_000]))
    assert False, "Invalid option"

SIZE_SPEC = [1,2,3,4,5]
def data_spec(opt):
    if opt == 1:
        return 50
    if opt == 2:
        return int(np.random.choice(range(100, 501, 100)))
    if opt == 3:
        return int(np.random.choice(range(200, 1501, 100)))
    if opt == 4:
        return int(np.random.choice(range(500, 4501, 100)))
    if opt == 5:
        return int(np.random.choice(range(1500, 4501, 100)))
    assert False, "Invalid option"

DEADLINE_SPEC = [1,2,3,4,5]
def deadline_spec(opt):
    if opt == 1:
        assert False
    if opt == 2:
        return int(np.random.choice([100_000, 200_000, 400_000, 800_000, 1_600_000]))
    if opt == 3:
        return int(np.random.choice([10_000, 25_000, 50_000, 100_000, 200_000, 400_000]))
    if opt == 4:
        return int(np.random.choice([0, 10_000, 20_000, 25_000, 50_000]))
    if opt == 5:
        return 0
    assert False, "Invalid option"


NUM_PORT = 4
NUM_QUEUES = 8
DATA_RATE = 1
ERROR = 1_000
STRUCTURE = 1     ## 0:LINEAR, 1:mesh, 2:MESH, 3:INDUSTRIAL
NUM_SW = 8
NUM_ES = 8
NUM_NODE = 16


In [3]:
def bfs_paths(graph, start, goal):
    return nx.shortest_path(graph, start, goal)

In [4]:
# def network(topo, header):
#     net = np.zeros(shape = (NUM_NODE, NUM_NODE))

#     ## Liner topology
#     if topo >= 0:
#         ## Connect line
#         for i in range(0, NUM_SW - 1):
#             net[i, i+1] = 1
#             net[i+1, i] = 1
#         ## Connect ring
#         if topo >= 1:
#             net[0, NUM_SW - 1] = 1
#             net[NUM_SW - 1, 0] = 1
#         if topo >= 2:
#             for i in range(0, NUM_SW // 2):
#                 net[i, NUM_SW - i - 1] = 1
#                 net[NUM_SW - i - 1, i] = 1

#     result = []
#     for i in range(NUM_SW):
#         for k in range(NUM_SW):
#             if net[i][k]:
#                 link = []
#                 link.append((i, k))
#                 link.append(NUM_QUEUES)
#                 link.append(DATA_RATE)
#                 link.append(ERROR)
#                 link.append(0)
#                 result.append(link)
                
#         net[i+NUM_SW, i] = 1
#         link = []
#         link.append((i + NUM_SW, i))
#         link.append(NUM_QUEUES)
#         link.append(DATA_RATE)
#         link.append(ERROR)
#         link.append(0)
#         result.append(link)
        
#         net[i, i+NUM_SW] = 1
#         link = []
#         link.append((i, i + NUM_SW))
#         link.append(NUM_QUEUES)
#         link.append(DATA_RATE)
#         link.append(ERROR)
#         link.append(0)
#         result.append(link)

#     result = pd.DataFrame(result, columns=['link','q_num','rate','t_proc','t_prop'])
#     result.to_csv(header + '_topology.csv', index=False)
    
#     return net

In [5]:
def line(num_sw, num_queue, data_rate, header):
    num_node = num_sw * 2
    net = np.zeros(shape = (num_node, num_node))

    ## Connect the line
    for i in range(0, num_sw - 1):
        net[i, i+1] = 1
        net[i+1, i] = 1
    ## Connect the switch and the end-station
    for i in range(num_sw):
        net[i+num_sw, i] = 1
        net[i, i+num_sw] = 1

    result = []
    for i in range(num_node):
        for j in range(num_node):
            if net[i][j]:
                link = []
                link.append((i, j))
                link.append(num_queue)
                link.append(data_rate)
                link.append(ERROR)
                link.append(0)
                result.append(link)

    result = pd.DataFrame(result, columns=['link','q_num','rate','t_proc','t_prop'])
    result.to_csv(header + '.csv', index=False)
    return net

In [6]:
def ring(num_sw, num_queue, data_rate, header):
    num_node = num_sw * 2
    net = np.zeros(shape = (num_node, num_node))

    ## Connect the line
    for i in range(0, num_sw - 1):
        net[i, i+1] = 1
        net[i+1, i] = 1
    ## Connect the switch and the end-station
    for i in range(num_sw):
        net[i+num_sw, i] = 1
        net[i, i+num_sw] = 1
    
    ## Connect the ring
    net[0, num_sw - 1] = 1
    net[num_sw - 1, 0] = 1

    result = []
    for i in range(num_node):
        for j in range(num_node):
            if net[i][j]:
                link = []
                link.append((i, j))
                link.append(num_queue)
                link.append(data_rate)
                link.append(ERROR)
                link.append(0)
                result.append(link)

    result = pd.DataFrame(result, columns=['link','q_num','rate','t_proc','t_prop'])
    result.to_csv(header + '.csv', index=False)
    return net

In [7]:
def tree(num_sw, num_queue, data_rate, header):
    num_node = num_sw * 2 + 1
    net = np.zeros(shape = (num_node, num_node))

    for i in range(num_sw):
        net[i, i * 2 + 1] = 1
        net[i * 2 + 1, i] = 1
        net[i, i * 2 + 2] = 1
        net[i * 2 + 2, i] = 1
    

    # for level in range(np.log2(num_node + 1).astype(int) - 1):
    #     for sw in range(int(2**level - 1), int(2**level - 1 + 2**(level))):
    #         net[sw, sw * 2 + 1] = 1
    #         net[sw * 2 + 1, sw] = 1
    #         net[sw, sw * 2 + 2] = 1
    #         net[sw * 2 + 2, sw] = 1
    
    # rest_nodes = num_node - 2**np.log2(num_node + 1).astype(int)
    # for i in range(rest_nodes):
    #     level = np.log2(num_node + 1).astype(int) - 1
    #     sw = int(2**level - 1) + i
    #     net[sw, sw * 2 + 1] = 1
    #     net[sw * 2 + 1, sw] = 1

    result = []
    for i in range(num_node):
        for j in range(num_node):
            if net[i][j]:
                link = []
                link.append((i, j))
                link.append(num_queue)
                link.append(data_rate)
                link.append(ERROR)
                link.append(0)
                result.append(link)

    result = pd.DataFrame(result, columns=['link','q_num','rate','t_proc','t_prop'])
    result.to_csv(header + '.csv', index=False)
    return net

In [8]:
def mesh(num_sw, num_queue, data_rate, header):
    num_node = num_sw * 2
    net = np.zeros(shape = (num_node, num_node))
    
    ## Connect the line
    for i in range(0, num_sw - 1):
        net[i, i+1] = 1
        net[i+1, i] = 1
    ## Connect the switch and the end-station
    for i in range(num_sw):
        net[i+num_sw, i] = 1
        net[i, i+num_sw] = 1
    
    ## Connect the mesh
    net[0, num_sw - 1] = 1
    net[num_sw - 1, 0] = 1

    ## Connect sw on the ring like DNA
    for i in range(0, num_sw // 2):
        net[i, num_sw - i - 1] = 1
        net[num_sw - i - 1, i] = 1

    result = []
    for i in range(num_node):
        for j in range(num_node):
            if net[i][j]:
                link = []
                link.append((i, j))
                link.append(num_queue)
                link.append(data_rate)
                link.append(ERROR)
                link.append(0)
                result.append(link)

    result = pd.DataFrame(result, columns=['link','q_num','rate','t_proc','t_prop'])
    result.to_csv(header + '.csv', index=False)
    return net

In [9]:
topo_func = [line, ring, tree, mesh]

In [10]:
# def workload(uti_thres, net, header):
#     result = []
#     period_opt = np.random.choice(PERIOD_SPEC)
#     size_opt = np.random.choice(SIZE_SPEC)
#     deadline_opt = np.random.choice(DEADLINE_SPEC)
#     i = 0
#     uti = 0
#     MAX_FLOW = 512
#     uti_ports = np.zeros(NUM_ES)
#     while True:
#         if uti / NUM_ES >= uti_thres:
#             result = pd.DataFrame(result, columns = ['id','src','dst','size','period','deadline','jitter'])
#             result.to_csv(header + '.csv', index=False)
#             return

#         availble_es = np.argwhere(uti_ports <= 0.75).reshape(-1)
#         if availble_es.size == 0 or i > MAX_FLOW:
#             i = 0
#             uti = 0
#             uti_ports = np.zeros(NUM_ES)
#             period_opt = np.random.choice(PERIOD_SPEC)
#             size_opt = np.random.choice(SIZE_SPEC)
#             deadline_opt = np.random.choice(DEADLINE_SPEC)
#             result = []
#             continue
#         start = int(np.random.choice(availble_es + NUM_SW))
#         end = int(np.random.choice([x for x in range(NUM_SW, NUM_SW + NUM_ES) if x != start]))
#         path = bfs_paths(graph, start, end)

#         period = period_spec(period_opt)
#         size = data_spec(size_opt)
#         deadline = (len(path) - 1) * (ERROR + size * 8) + deadline_spec(deadline_opt) if deadline_opt > 1 else period
#         if deadline <= period:
#             result.append([i, start, [end], size, period, deadline, deadline])
#             uti += size * 8 / period
#             uti_ports[start - NUM_SW] += size * 8 / period
#             i += 1
#         else:
#             continue
        
# HEADER = 'utilization'
# for ins in tqdm(range(512)):
#     for util_thres in np.arange(0.05, 0.45, 0.05):
#         header =  HEADER + '/' + HEADER + '_' + str(int(util_thres * 100)) + '_' + str(ins)
#         net = topo_func[np.random.randint(0, len(topo_func))](num_sw = 8, num_queue = 8, data_rate = 1, header = header + '_topo')
#         graph = nx.from_numpy_matrix(net)
#         workload(util_thres, graph, header=header + '_task')
        

In [11]:
# def streams(num_thres, net, header):
#     result = []
#     period_opt = np.random.choice(PERIOD_SPEC)
#     size_opt = np.random.choice(SIZE_SPEC)
#     deadline_opt = np.random.choice(DEADLINE_SPEC)
#     i = 0
#     uti = 0
#     uti_ports = np.zeros(NUM_ES)
#     while True:
#         if i >= num_thres:
#             result = pd.DataFrame(result, columns = ['id','src','dst','size','period','deadline','jitter'])
#             result.to_csv(header + '.csv', index=False)
#             return

#         availble_es = np.argwhere(uti_ports <= 0.75).reshape(-1)
#         if availble_es.size == 0:
#             availble_es = np.array([x for x in range(NUM_ES)])
            
#         start = int(np.random.choice(availble_es + NUM_SW))
#         end = int(np.random.choice([x for x in range(NUM_SW, NUM_SW + NUM_ES) if x != start]))
#         path = bfs_paths(graph, start, end)

#         period = period_spec(period_opt)
#         size = data_spec(size_opt)
#         deadline = (len(path) - 1) * (ERROR + size * 8) + deadline_spec(deadline_opt) if deadline_opt > 1 else period
#         if deadline <= period:
#             result.append([i, start, [end], size, period, deadline, deadline])
#             uti += size * 8 / period
#             uti_ports[start - NUM_SW] += size * 8 / period
#             i += 1
#         else:
#             continue
        

# HEADER = 'stream'
# for ins in tqdm(range(512)):
#     for num_thres in range(8, 189, 10):
#         header = HEADER + '/' + HEADER + '_' + str(num_thres) + '_' + str(ins)
#         net = topo_func[np.random.randint(0, len(topo_func))](num_sw = 8, num_queue = 8, data_rate = 1, header = header + '_topo')
#         graph = nx.from_numpy_matrix(net)
#         streams(num_thres, graph, header=header + '_task')

In [12]:
# def period_ins(period_opt, net, header):
#     result = []
#     size_opt = np.random.choice(SIZE_SPEC)
#     deadline_opt = np.random.choice(DEADLINE_SPEC)
#     num_thres = np.random.choice(NUM_STREAM_SPEC)
#     i = 0
#     uti = 0
#     uti_ports = np.zeros(NUM_ES)
#     while True:
#         if i >= num_thres:
#             result = pd.DataFrame(result, columns = ['id','src','dst','size','period','deadline','jitter'])
#             result.to_csv(header + '.csv', index=False)
#             return

#         availble_es = np.argwhere(uti_ports <= 0.75).reshape(-1)
#         if availble_es.size == 0:
#             availble_es = np.array([x for x in range(NUM_ES)])

#         availble_es = np.array([x for x in range(NUM_ES)])
#         start = int(np.random.choice(availble_es + NUM_SW))
#         end = int(np.random.choice([x for x in range(NUM_SW, NUM_SW + NUM_ES) if x != start]))
#         path = bfs_paths(graph, start, end)

#         period = period_spec(period_opt)
#         size = data_spec(size_opt)
#         deadline = (len(path) - 1) * (ERROR + size * 8) + deadline_spec(deadline_opt) if deadline_opt > 1 else period
#         if deadline <= period:
#             result.append([i, start, [end], size, period, deadline, deadline])
#             uti += size * 8 / period
#             uti_ports[start - NUM_SW] += size * 8 / period
#             i += 1
#         else:
#             continue
        

# HEADER = 'period'
# for ins in tqdm(range(512)):
#     for period_opt in PERIOD_SPEC:
#         header = HEADER + '/' + HEADER + '_' + str(period_opt) + '_' + str(ins)
#         net = topo_func[np.random.randint(0, len(topo_func))](num_sw = 8, num_queue = 8, data_rate = 1, header = header + '_topo')
#         graph = nx.from_numpy_matrix(net)
#         period_ins(period_opt, graph, header + '_task')

In [13]:
# def payload(size_opt, net, header):
#     result = []
#     period_opt = np.random.choice(PERIOD_SPEC)
#     deadline_opt = np.random.choice(DEADLINE_SPEC)
#     num_thres = np.random.choice(NUM_STREAM_SPEC)
#     i = 0
#     uti = 0
#     uti_ports = np.zeros(NUM_ES)
#     while True:
#         if i >= num_thres:
#             result = pd.DataFrame(result, columns = ['id','src','dst','size','period','deadline','jitter'])
#             result.to_csv(header + '.csv', index=False)
#             return

#         availble_es = np.argwhere(uti_ports <= 0.75).reshape(-1)
#         if availble_es.size == 0:
#             availble_es = np.array([x for x in range(NUM_ES)])

#         availble_es = np.array([x for x in range(NUM_ES)])
#         start = int(np.random.choice(availble_es + NUM_SW))
#         end = int(np.random.choice([x for x in range(NUM_SW, NUM_SW + NUM_ES) if x != start]))
#         path = bfs_paths(graph, start, end)

#         period = period_spec(period_opt)
#         size = data_spec(size_opt)
#         deadline = (len(path) - 1) * (ERROR + size * 8) + deadline_spec(deadline_opt) if deadline_opt > 1 else period
#         if deadline <= period:
#             result.append([i, start, [end], size, period, deadline, deadline])
#             uti += size * 8 / period
#             uti_ports[start - NUM_SW] += size * 8 / period
#             i += 1
#         else:
#             continue
        

# HEADER = 'payload'
# for ins in tqdm(range(512)):
#     for payload_opt in SIZE_SPEC:
#         header = HEADER + '/' + HEADER + '_' + str(payload_opt) + '_' + str(ins)
#         net = topo_func[np.random.randint(0, len(topo_func))](num_sw = 8, num_queue = 8, data_rate = 1, header = header + '_topo')
#         graph = nx.from_numpy_matrix(net)
#         payload(payload_opt, graph, header + '_task')
        

In [14]:
# def deadline(deadline_opt, net, header):
#     result = []
#     period_opt = np.random.choice(PERIOD_SPEC)
#     size_opt = np.random.choice(SIZE_SPEC)
#     num_thres = np.random.choice(NUM_STREAM_SPEC)
#     i = 0
#     uti = 0
#     uti_ports = np.zeros(NUM_ES)
#     while True:
#         if i >= num_thres:
#             result = pd.DataFrame(result, columns = ['id','src','dst','size','period','deadline','jitter'])
#             result.to_csv(header + '.csv', index=False)
#             return

#         availble_es = np.argwhere(uti_ports <= 0.75).reshape(-1)
#         if availble_es.size == 0:
#             availble_es = np.array([x for x in range(NUM_ES)])

#         availble_es = np.array([x for x in range(NUM_ES)])
        
#         start = int(np.random.choice(availble_es + NUM_SW))
#         end = int(np.random.choice([x for x in range(NUM_SW, NUM_SW + NUM_ES) if x != start]))
#         path = bfs_paths(graph, start, end)

#         period = period_spec(period_opt)
#         size = data_spec(size_opt)
#         deadline = (len(path) - 1) * (ERROR + size * 8) + deadline_spec(deadline_opt) if deadline_opt > 1 else period
#         if deadline <= period:
#             result.append([i, start, [end], size, period, deadline, deadline])
#             uti += size * 8 / period
#             uti_ports[start - NUM_SW] += size * 8 / period
#             i += 1
#         else:
#             continue
        

# HEADER = 'deadline'
# for ins in tqdm(range(512)):
#     for deadline_opt in DEADLINE_SPEC:
#         header = HEADER + '/' + HEADER + '_' + str(deadline_opt) + '_' + str(ins)
#         net = topo_func[np.random.randint(0, len(topo_func))](num_sw = 8, num_queue = 8, data_rate = 1, header = header + '_topo')
#         graph = nx.from_numpy_matrix(net)
#         deadline(deadline_opt, graph, header + '_task')



# # HEADER = 'utilization'
# # for ins in tqdm(range(512)):
# #     for util_thres in np.arange(0.05, 0.45, 0.05):
# #         header =  HEADER + '/' + HEADER + '_' + str(int(util_thres * 100)) + '_' + str(ins)
# #         net = topo_func[np.random.randint(0, len(topo_func))](num_sw = 8, num_queue = 8, data_rate = 1, header = header + '_topo')
# #         graph = nx.from_numpy_matrix(net)
# #         workload(util_thres, graph, header=header + '_task')

In [15]:
## Check whether the generated deadline are different
# for deadline_opt in DEADLINE_SPEC:
#     average_deadline = []
#     for ins in range(512):
#         df = pd.read_csv('deadline/deadline_' + str(deadline_opt) + '_' + str(ins) + '.csv')
#         average_deadline.append(df['deadline'].mean())
#     print(deadline_opt, np.mean(average_deadline))

# Deadline is different
# 1 1064098.9878821163
# 2 407226.02442196634
# 3 158710.73453289864
# 4 72887.87741810564
# 5 50997.09877900492

In [16]:
def topo(graph, header, rate, num_es, num_sw):
    result = []
    deadline_opt = np.random.choice(DEADLINE_SPEC)
    period_opt = np.random.choice(PERIOD_SPEC)
    size_opt = np.random.choice(SIZE_SPEC)
    num_thres = np.random.choice(NUM_STREAM_SPEC)

    i = 0
    while True:
        if i >= num_thres:
            result = pd.DataFrame(result, columns = ['id','src','dst','size','period','deadline','jitter'])
            result.to_csv(header + '.csv', index=False)
            return

        start = int(np.random.choice(range(num_sw, num_sw + num_es)))
        end = int(np.random.choice([x for x in range(num_sw, num_sw + num_es) if x != start]))
        path = bfs_paths(graph, start, end)
        period = period_spec(period_opt)
        size = data_spec(size_opt)
        deadline = (len(path) - 1) * (ERROR + size * 8 / rate) + deadline_spec(deadline_opt) if deadline_opt > 1 else period
        if deadline <= period:
            result.append([i, start, [end], size, period, deadline, deadline])
            i += 1
        else:
            continue
        
        
# rate = 1

# HEADER = 'line'
# for num_sw in range(8, 98, 10):
#     num_es = num_sw
#     for ins in tqdm(range(512)):
#         header =  HEADER + '/' + HEADER + '_' + str(num_sw) + '_' + str(ins)
#         net = line(num_sw = num_sw, num_queue = 8, data_rate = 1, header = header + '_topo')
#         graph = nx.from_numpy_matrix(net)
#         topo(graph, header + '_task',rate, num_es, num_sw)
    

# HEADER = 'ring'
# for num_sw in range(8, 98, 10):
#     num_es = num_sw
#     for ins in tqdm(range(512)):
#         header =  HEADER + '/' + HEADER + '_' + str(num_sw) + '_' + str(ins)
#         net = ring(num_sw = num_sw, num_queue = 8, data_rate = 1, header = header + '_topo')
#         graph = nx.from_numpy_matrix(net)
#         topo(graph, header + '_task',rate, num_es, num_sw)

# HEADER = 'tree'
# for num_sw in range(8, 98, 10):
#     num_es = num_sw + 1
#     for ins in tqdm(range(512)):
#         header =  HEADER + '/' + HEADER + '_' + str(num_sw) + '_' + str(ins)
#         net = tree(num_sw = num_sw, num_queue = 8, data_rate = 1, header = header + '_topo')
#         graph = nx.from_numpy_matrix(net)
#         topo(graph, header + '_task',rate, num_es, num_sw)
        
# HEADER = 'mesh'
# for num_sw in range(8, 98, 10):
#     num_es = num_sw
#     for ins in tqdm(range(512)):
#         header =  HEADER + '/' + HEADER + '_' + str(num_sw) + '_' + str(ins)
#         net = mesh(num_sw = num_sw, num_queue = 8, data_rate = 1, header = header + '_topo')
#         graph = nx.from_numpy_matrix(net)
#         topo(graph, header + '_task', rate, num_es, num_sw)

In [17]:
# num_sw = 8
# num_es = 8
# HEADER = 'queue'

# for num_queue in range(1, 9):
#     for ins in tqdm(range(512)):
#         header =  HEADER + '/' + HEADER + '_' + str(num_queue) + '_' + str(ins)
#         net = topo_func[np.random.randint(0, len(topo_func))](num_sw, num_queue = num_queue, data_rate=1, header=header + '_topo')
#         graph = nx.from_numpy_matrix(net)
#         topo(graph, header + '_task', 1, num_es, num_sw)

In [23]:
num_sw = 8
num_es = 8
HEADER = 'rate'

for rate in [0.01, 0.1, 1, 10]:
    for ins in tqdm(range(512)):
        header =  HEADER + '/' + HEADER + '_' + str(int(rate * 1000)) + '_' + str(ins)
        net = topo_func[np.random.randint(0, len(topo_func))](num_sw, num_queue = 8, data_rate=rate, header=header + '_topo')
        graph = nx.from_numpy_matrix(net)
        topo(graph, header + '_task', 1, num_es, num_sw)

100%|██████████| 512/512 [00:03<00:00, 132.79it/s]
